## External imports

In [1]:
import os
import pandas as pd
import numpy as np

## Internal libraries imports

In [2]:
import wntr_WSN
import networkx_graph as ng
import linear_regression as lr

## Input data

In [3]:
method = 'method'                              # for file naming
leak_node = 'J64'                             # node that's leaking
comment = ''                                   # last word(s) in output filename
leak_start = 48                                # in hours
leak_end = 73                                  # in hours
leak_area = 0.0005
sim_duration = 130                             # total duration of the simulation

## Model each node with linear regression

In [4]:
linreg_models, normalization_params = lr.model_network_with_linreg(n=3)

Modeling the network with linreg (linear_regression.py - model_network_with_linregression())
Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())
Running the simulation (wntr_WSN.py - get_sim_results())


C:\Users\wij\Desktop\Inżynierka\WTNR-pipe-leak-detection-main\virWNTR\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


	Modeling junction J1: (1 out of 309)
	Modeling junction J2: (2 out of 309)
	Modeling junction J3: (3 out of 309)
	Modeling junction J4: (4 out of 309)
	Modeling junction J5: (5 out of 309)
	Modeling junction J6: (6 out of 309)
	Modeling junction J7: (7 out of 309)
	Modeling junction J8: (8 out of 309)
	Modeling junction J9: (9 out of 309)
	Modeling junction J10: (10 out of 309)
	Modeling junction J11: (11 out of 309)
	Modeling junction J12: (12 out of 309)
	Modeling junction J13: (13 out of 309)
	Modeling junction J14: (14 out of 309)
	Modeling junction J15: (15 out of 309)
	Modeling junction J16: (16 out of 309)
	Modeling junction J17: (17 out of 309)
	Modeling junction J18: (18 out of 309)
	Modeling junction J19: (19 out of 309)
	Modeling junction J20: (20 out of 309)
	Modeling junction J21: (21 out of 309)
	Modeling junction J22: (22 out of 309)
	Modeling junction J24: (23 out of 309)
	Modeling junction J25: (24 out of 309)
	Modeling junction J26: (25 out of 309)
	Modeling junction

	Modeling junction J212: (203 out of 309)
	Modeling junction J213: (204 out of 309)
	Modeling junction J214: (205 out of 309)
	Modeling junction J216: (206 out of 309)
	Modeling junction J218: (207 out of 309)
	Modeling junction J219: (208 out of 309)
	Modeling junction J220: (209 out of 309)
	Modeling junction J223: (210 out of 309)
	Modeling junction J224: (211 out of 309)
	Modeling junction J225: (212 out of 309)
	Modeling junction J226: (213 out of 309)
	Modeling junction J227: (214 out of 309)
	Modeling junction J228: (215 out of 309)
	Modeling junction J229: (216 out of 309)
	Modeling junction J230: (217 out of 309)
	Modeling junction J231: (218 out of 309)
	Modeling junction J232: (219 out of 309)
	Modeling junction J233: (220 out of 309)
	Modeling junction J235: (221 out of 309)
	Modeling junction J236: (222 out of 309)
	Modeling junction J239: (223 out of 309)
	Modeling junction J240: (224 out of 309)
	Modeling junction J241: (225 out of 309)
	Modeling junction J242: (226 out 

In [5]:
linreg_models

[{'node': 'J1',
  'linreg': LinearRegression(),
  'msq': 5.637341226912175e-05,
  'r2': 0.9999686251938643,
  'sensors': ['J284', 'P9', 'P460']},
 {'node': 'J2',
  'linreg': LinearRegression(),
  'msq': 5.978548504308501e-06,
  'r2': 0.9999963379189123,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J3',
  'linreg': LinearRegression(),
  'msq': 3.4779785988136976e-06,
  'r2': 0.9999979014912568,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J4',
  'linreg': LinearRegression(),
  'msq': 1.1570697641537868e-05,
  'r2': 0.9999929627215727,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J5',
  'linreg': LinearRegression(),
  'msq': 3.666036758666608e-05,
  'r2': 0.9999764201230342,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J6',
  'linreg': LinearRegression(),
  'msq': 0.00022031214051558388,
  'r2': 0.9998898148577574,
  'sensors': ['P9', 'J79', 'P500']},
 {'node': 'J7',
  'linreg': LinearRegression(),
  'msq': 7.921957869642725e-05,
  'r2': 0.999950449668531,
  'sensors': ['

In [6]:
# simulate a leak scenario
sim_LEAK = wntr_WSN.get_sim_results_LEAK(leak_node, leak_area, leak_start, leak_end)

# join pressures and flowrates
sim_LEAK_raw = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)
predictions = {}

# for each node in graph
G = ng.create_graph()
for node in G.nodes():

    # find linear regression model for this particular node
    for model in linreg_models:
        if model['node'] == node:
            linreg_dict = model

    # choose only the available sensors
    sim_LEAK_scope = sim_LEAK_raw[linreg_dict['sensors']]

    # normalise the input data with respective parameters
    sim_LEAK_scope = (sim_LEAK_scope - normalization_params['mean'].loc[linreg_dict['sensors']]) / normalization_params['std'].loc[linreg_dict['sensors']]

    # predict pressure for each node
    predictions[node] = linreg_dict['linreg'].predict(sim_LEAK_scope)

# "sim_duration*6" because (60 minutes = 10 minutes * 6)
# it's related to simulation step time 
index = [600*i for i in range(0, (sim_duration*6)+1)]

pred_LEAK_results = pd.DataFrame(data=predictions, index=index)
pred_LEAK_results

C:\Users\wij\Desktop\Inżynierka\WTNR-pipe-leak-detection-main\virWNTR\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())


ValueError: Shape of passed values is (673, 309), indices imply (781, 309)

## Get results of leak scenario

In [7]:
sim_LEAK_results = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)
sim_LEAK_results

,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,...,P548,P549,P551,P553,PMP51,PMP63,PMP72,FCV51,FCV63,FCV72
0,36.011416,30.044852,29.924852,31.958688,32.148688,45.531102,31.841401,28.841400,33.861399,43.310270,...,0.005835,-0.001642,0.001183,-0.000037,0.020833,0.0,0.0434,0.020833,0.0,0.0434
3600,36.643471,30.803467,30.683467,32.673468,32.863468,45.903506,32.473456,29.473455,34.493454,43.604068,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
7200,36.438482,30.598478,30.478478,32.468479,32.658479,45.698517,32.268468,29.268467,34.288465,43.399079,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
10800,36.233493,30.393490,30.273490,32.263490,32.453490,45.493529,32.063479,29.063478,34.083477,43.194090,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
14400,36.028505,30.188501,30.068501,32.058502,32.248502,45.288540,31.858490,28.858489,33.878488,42.989102,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404800,35.267360,29.429958,29.309958,31.299045,31.489045,44.523787,31.097036,28.097001,33.116992,42.223793,...,0.002562,-0.000556,-0.000079,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
2408400,35.057673,29.223796,29.103796,31.091653,31.281653,44.307205,30.887490,27.887455,32.907477,42.006064,...,0.001842,-0.000397,-0.000146,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
2412000,34.753043,28.920796,28.800796,30.788086,30.978086,44.000091,30.582747,27.582680,32.602734,41.698375,...,0.001672,-0.000357,-0.000163,-0.000022,0.000000,0.0,0.0000,0.000000,0.0,0.0000
2415600,34.448502,28.605903,28.485903,30.476792,30.666792,43.714177,30.278488,27.278486,32.298485,41.417237,...,0.002539,-0.000558,0.000203,-0.000009,0.000000,0.0,0.0000,0.000000,0.0,0.0000


## Calculate residuum

In [8]:
residuals = sim_LEAK.node['pressure'] - pred_LEAK_results
residuals

NameError: name 'pred_LEAK_results' is not defined

## Take care of column naming convention

In [9]:
residuals = residuals.add_suffix('_residuum')

sim_LEAK.node['pressure'] = sim_LEAK.node['pressure'].add_suffix('_pressure')
sim_LEAK.link['flowrate'] = sim_LEAK.link['flowrate'].add_suffix('_flow')
sim_LEAK = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)

pred_LEAK_results = pred_LEAK_results.add_suffix('_model_output')

NameError: name 'residuals' is not defined

## Sanity check

In [ ]:
print(f'Number of residuals columns: {len(residuals.columns)}')
print(f'Number of sim_LEAK columns: {len(sim_LEAK.columns)}')
print(f'Number of pred_LEAK_results columns: {len(pred_LEAK_results.columns)}')
print(f'Sum: {len(residuals.columns) + len(sim_LEAK.columns) + len(pred_LEAK_results.columns)}')

## Form final dataframe

In [ ]:
output = pd.concat([residuals, sim_LEAK, pred_LEAK_results], axis=1)
output

## Export the final dataframe to .xlsx

In [ ]:
if comment:
    output.to_excel(os.path.join(os.getcwd(), f'{method}_n{leak_node}_la{leak_area}_lst{leak_start}_let{leak_end}_{comment}.xlsx'))
else:
    output.to_excel(os.path.join(os.getcwd(), f'{method}_n{leak_node}_la{leak_area}_lst{leak_start}_let{leak_end}.xlsx'))